In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css : string = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Evaluating an Exam Using TypeScript

This notebook shows how we can use TypeScript regular expressions to implement a scanner. Our goal is to implement a program that can be used to evaluate the results of an exam. Assume the result of an exam is stored in the string `data` that is defined below:

In [ ]:
const data: string = `Class: Algorithms and Complexity
          Group: TINF22AI1
          MaxPoints = 60
   
          Exercise:      1. 2. 3. 4. 5. 6.
          Jim Smith:     9 12 10  6  6  0
          John Slow:     4  4  2  0  -  -
          Susi Sorglos:  9 12 12  9  9  6
          1609922:       7  4 12  5  5  3
       `;

This data show that there has been a exam with the subject <em style="color:blue">Algorithms and Complexity</em>
in the group <em style="color:blue">TINF22AI1</em>.  Furthermore, the equation
```
   MaxPoints = 60
```
shows that in order to achieve the best mark, <em style="color:blue">60</em> points would have been necessary.

There have been 6 different exercises in this exam and, in this small example,  only four students took part, namely *Jim Smith*, *John Slow*, *Susi Sorglos*, and some student that is only represented by their matriculation number.  Each of the rows decribing the results of the students begins with the name (or matriculation number) of the student followed by the number of points that they have achieved in the different exercises. Our goal is to write a program that is able to compute the marks for all students.

## Auxiliary Functions

The function `mark(maxPoints: number, points: number): number` takes two arguments and returns a numeric grade:

**Parameters:**
- `maxPoints: number` - The number of points needed to achieve the best mark of 1.0
- `points: number` - The number of points achieved by the student

**Return value:**
- `number` - The calculated grade (between 1.0 and 5.0)

It is assumed that the relation between the mark and the points is mostly linear. A student who achieves 50% of `maxPoints` will get the mark 4.0, while 100% results in mark 1.0.

The formula to calculate the grade is:
$$ \textrm{grade} = 7 - 6 \cdot \frac{\texttt{points}}{\texttt{maxPoints}} $$

However, the worst mark is 5.0. The `Math.min()` function ensures the grade does not exceed 5.0. The result is rounded to one decimal place using `Math.round()`.

In [ ]:
function mark(maxPoints: number, points: number): number {
    if (maxPoints === 0) return 0;
    const grade = 7 - (6 * points) / maxPoints;
    return Math.round(Math.min(5.0, grade) * 10) / 10;
}

## Visualizing the Grading Function

To better understand how our `mark()` function converts points to grades, let's visualize it:

In [ ]:
import { plotGradeFunction } from "./utils/plotGrade";

plotGradeFunction(mark, 60);

The resulting plot shows how the grade decreases linearly from 5.0 (worst) at 0 points to 1.0 (best) at 60 points, with a grade of 4.0 achieved at exactly 50% of the maximum points (30 points).

## Regular Expression Definitions

We use TypeScript's built-in regular expressions to identify the different parts of the input data.

We define the following regex patterns:

- `MAXDEF`: Matches the line defining the maximum points (e.g., `MaxPoints = 60`).
- `MATRICULATION`: Matches a student identified by a 7-digit number followed by a colon.
- `NAME`: Matches a student name (letters, spaces, or hyphens) followed by a colon.
- `HEADER`: Matches header lines like "Class:" or "Group:". Note that we distinguish these from names by assuming student names contain spaces (e.g., "Jim Smith") while headers in this format are single words before the colon.
- `NUMBER`: Matches the points awarded for each exercise.

In [ ]:
const MAXDEF_REGEX        = /MaxPoints\s*=\s*([1-9][0-9]*)/;
const MATRICULATION_REGEX = /^([0-9]{7}):/;
const NAME_REGEX          = /^([A-Za-z -]+):/;
const NUMBER_REGEX        = /0|[1-9][0-9]*/g;

## Scanner Implementation

### Parsing the Maximum Points

In the implementation of a scanner, we often need to extract configuration values that affect how subsequent data is interpreted. In our exam data, the `MaxPoints` definition acts as a state change.

The function `parseMaxPoints` acts as a specialized matcher. It attempts to extract the integer value solely if the line corresponds to a configuration definition.

**Specs:**
* **Input:** The current line string.
* **Output:** The integer points if found, otherwise `undefined`.

In [ ]:
function parseMaxPoints(line: string): number | undefined {
    const match = line.match(MAXDEF_REGEX);
    if (match && match[1]) return parseInt(match[1], 10);
    return undefined;
}

### Identifying Students (Disambiguation)

A core challenge in scanning unstructured text is **disambiguation**. A line starting with text could be a student's name, a matriculation number, or a header label.

The function `parseStudentName` implements a priority logic to identify a student:
1.  **Matriculation Check:** A 7-digit number (`MATRICULATION_REGEX`) is unambiguous.
2.  **Name Check:** If it looks like a name (`NAME_REGEX`), we apply a **heuristic**: valid student names must contain a space (First Last). Single-word labels like "Exercise:" are rejected.

**Specs:**
* **Input:** The line to analyze.
* **Output:** The identifier (name or ID) as a string, or `undefined`.

In [ ]:
function parseStudentName(line: string): string | undefined {
    const matMatch = line.match(MATRICULATION_REGEX);
    if (matMatch && matMatch[1]) return matMatch[1];

    const nameMatch = line.match(NAME_REGEX);
    if (nameMatch && nameMatch[1]) {
        const label = nameMatch[1];
        if (label.includes(' ')) return label;
    }
    return undefined;
}

### Tokenizing and Aggregating Scores

Once a line is identified as a student record, the scanner shifts to **tokenizing** the scores.

The function `calculateLineScore` ignores the prefix (student name) and focuses on the numeric tokens. It splits the line, extracts all number sequences via Regex, and reduces them to a sum.

**Specs:**
* **Input:** The full line containing student info and points.
* **Output:** The sum of all points (0 if malformed).

In [ ]:
function calculateLineScore(line: string): number {
    const parts = line.split(':');
    if (parts.length < 2) return 0;

    const scoresPart = parts.slice(1).join(':');
    const matches = scoresPart.match(NUMBER_REGEX);
    
    if (!matches) return 0;
    return matches.reduce((sum, numStr) => sum + parseInt(numStr, 10), 0);
}

### The Main Scanner Loop

Finally, the `evaluateExam` function acts as the **Controller**. It iterates through the stream and manages the **Program State**.

The logic follows a strict precedence (Guard Clauses) to handle the input efficiently:
1.  **Skip Empty Lines:** Noise reduction.
2.  **Update State:** If a `MaxPoints` definition is found, the context (`currentMaxPoints`) is updated for subsequent lines.
3.  **Process Student:** If a student is identified, the score is calculated using the current context.

In [ ]:
function evaluateExam(inputData: string): void {
    const lines = inputData.split('\n');
    let currentMaxPoints = 0;

    for (const rawLine of lines) {
        const line = rawLine.trim();
        
        // 1. Skip noise
        if (line.length === 0) continue;

        // 2. Check for state change
        const newMaxPoints = parseMaxPoints(line);
        if (newMaxPoints !== undefined) {
            currentMaxPoints = newMaxPoints;
            continue;
        }

        // 3. Check for data entry
        const studentName = parseStudentName(line);
        if (!studentName) continue;

        // 4. Process data
        const sumPoints = calculateLineScore(line);
        const grade = mark(currentMaxPoints, sumPoints);
        console.log(`${studentName} has ${sumPoints} points and achieved the mark ${grade}.`);
    }
}

## Running the Scanner

Finally, we run our evaluation function on the data provided.

In [ ]:
evaluateExam(data);